In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [169]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Problem Statement
How might we predict a country’s **protein supply per capita** through the analysis of social, environmental, and economic factors such as **GDP, temperature change, consumer prices, import and export quantity and production quantity**?

# Overall 7 steps Methodology for ML Project
Reference: [Google's 7 steps of ML](https://towardsdatascience.com/the-googles-7-steps-of-machine-learning-in-practice-a-tensorflow-example-for-structured-data-96ccbb707d77)

Step 1: [Gathering Data](#step1)    
Step 2: [Data Preparation](#step2)  
Step 3: [Choosing a model](#step3)  
Step 4: [Training](#step4)  
Step 5: [Evaluation](#step5)  
Step 6: [Parameter Tuning](#step6)  
Step 7: [Prediction](#step7)  

Step 0: [Model Improvements](#modelimp)


<a id='step1'></a>
# Step 1: Gathering Data (Copi from Math docs)
  
Our data is sourced from FAOstats \<insert link>  
  
Before selecting our predictor values, we first designed a persona for each of our statistics. This is done below in [Step 2: data preparation](#step2) through the use of graphs to map out potentially interesting characteristics of each country in relation to each predictor.  

We have summarized our findings in the table below:  
\<Insert table of persona in markdown>

<a id='step2'></a>
# Step 2: Data Preparation
1. Aggregation of data by countries (e.g averaging temperature changes, summing production/consumption over the year and averaging by population)
    * Ensure all data able to be compared fairly  
    

2. Rename all countries to a consistent naming scheme. This is just in case of different source materials.  


3. Preprocessing - Drop Null Rows and Missing Data
    * Save as new csv file, Data_aggregated.csv


4. Visualize data and relationships of protein supply against various X using seaborn and matplotlib using scatter plots
    * To check if there are any obvious relationships that can be seen
    * Persona Analysis by observing distribution of datas for each X


5. Normalize data with Z-normalization
    * Save as new csv, Data_norm.csv
    

In [170]:
def normalize_z(dfin):
    dfout = pd.DataFrame()
    mean = dfin.mean(axis=0) 
    std = dfin.std(axis=0)  
    dfout = ((dfin - mean)/std)
    return dfout

def get_features_targets(df, feature_names, target_names):
    df_feature = df[feature_names]
    df_target = df[target_names]
    return df_feature, df_target

def prepare_feature(df_feature):
    # this is to convert table of x independent variables to X matrix, hence first column is all ones as x_0=1
    np_feature = df_feature.to_numpy()
    np_m = np_feature.shape[0]
    if np_feature.ndim == 1: 
        np_feature = np_feature.reshape(np_m, 1)
    big_X = np.concatenate((np.ones((np_m, 1), dtype=float), np_feature), axis=1)
    return big_X

def prepare_target(df_target):
    # this is for the y dependent variable to be predicted
    np_target = df_target.to_numpy()
    return np_target

def predict(df_feature, beta):
    norm_feature = normalize_z(df_feature) 
    np_feature = prepare_feature(norm_feature) 
    y_hat = calc_linear(np_feature, beta)  
    return y_hat

def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    # df.sample and df.drop can also do this 
    np.random.seed(random_state)
    
    df_feature_rows = df_feature.shape[0]
    df_target_rows = df_target.shape[0]
    
    df_feature_split = int(test_size * df_feature_rows)
    df_target_split = int(test_size * df_target_rows)
    
    df_feature_rando = np.random.choice(df_feature_rows, size=df_feature_rows, replace=False)  
    df_feature_test = df_feature.iloc[df_feature_rando[:df_feature_split]]   # split the randomized index and use to get values in the idx rows
    df_feature_train = df_feature.iloc[df_feature_rando[df_feature_split:]]
    
    df_target_test = df_target.iloc[df_feature_rando[:df_target_split]]
    df_target_train = df_target.iloc[df_feature_rando[df_target_split:]]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test

In [171]:
# Read CSVs
import os

csv_dir = os.path.join(os.getcwd(), "Datas", "Datas", "CSV")

# Population
df_pop = pd.read_csv(os.path.join(csv_dir, "population.csv"))

# Protein Supply per g per pax (y)
df_protein = pd.read_csv(os.path.join(csv_dir, "protein-supply.csv"))

# GDP per capita
df_gdp = pd.read_csv(os.path.join(csv_dir, "GDP_per_capita.csv"))

# Temperature change
df_temp = pd.read_csv(os.path.join(csv_dir, "temperature_change.csv"))

# Consumer prices
df_price = pd.read_csv(os.path.join(csv_dir, "consumer-prices.csv"))

# Import quantity
df_import = pd.read_csv(os.path.join(csv_dir, "import_quantity_raw.csv"))

# Export quantity
df_export = pd.read_csv(os.path.join(csv_dir, "export_quantity_raw.csv"))

# Production quantity
df_prod = pd.read_csv(os.path.join(csv_dir, "production_quantity.csv"))

In [172]:
# view data for preprocessing

# display(df_pop)
# display(df_import)
display(df_protein)
# display(df_gdp)
# df_pop["Value"].rename(df_pop["Area"])

,Product,Country,Year,Population,Production (t),production__tonnes__per_capita,Production per capita (kg),Yield (t/ha),Yield (kg/animal),Land Use (ha),...,other_uses__tonnes__per_capita,Other uses per capita (kg),Supply chain waste (t),waste_in_supply_chain__tonnes__per_capita,Supply chain waste per capita (kg),Food supply (kg per capita per year),Food supply (g per capita per day),Food supply (kcal per capita per day),Food supply (Protein g per capita per day),Food supply (Fat g per capita per day)
0,NaN,Afghanistan,1961,8790140,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3054.9053,86.492840,38.209236
1,NaN,Afghanistan,1962,8969055,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2973.2468,84.580055,38.335213
2,NaN,Afghanistan,1963,9157463,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2751.7795,78.657250,39.338820
3,NaN,Afghanistan,1964,9355510,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3013.4424,85.198880,39.747234
4,NaN,Afghanistan,1965,9565154,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3017.7600,85.612100,40.560085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12216,NaN,Zimbabwe,2015,14154937,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1872.9144,42.347970,55.084568
12217,NaN,Zimbabwe,2016,14452705,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1899.7627,42.004463,58.332520
12218,NaN,Zimbabwe,2017,14751101,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1797.1061,42.225240,50.178062
12219,NaN,Zimbabwe,2018,15052191,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1798.6170,41.094800,50.533940


In [174]:
# Aggregation of data by countries
# ---------------------------------

# Country Population per 1000
pop_series = df_pop["Value"].rename(df_pop["Area"])

# Mean of temperature change over months of 2019
# This dataset has more countries than the others
srs_temp_agg = df_temp.groupby("i")["Value"].mean()  # series with label as country

# Mean of Consumer Price Index over the year
srs_cpi_agg = df_price.groupby("Area")["Value"].mean()

# GDP per capita
srs_gdp_cap = pd.Series(data=df_gdp["Value"].values, index=df_gdp["Area"])

# Import Quantity in tonnes
srs_import_agg = df_import.groupby("Area")["Value"].sum()  # series


# Import Quantity in tonnes per 1000
srs_import_agg_1000 = srs_import_agg.divide(pop_series, axis=0)


# Export Quantity in tonnes
srs_export_agg = df_export.groupby("Area")["Value"].sum() 


# Export Quantity in tonnes per 1000
srs_export_agg_1000 = srs_export_agg.divide(pop_series, axis=0)


# Production Quantity in tonnes 
srs_prod_agg = df_prod.groupby("Area")["Value"].sum() 


# Production Quantity in tonnes per 1000
srs_prod_agg_1000 = srs_prod_agg.divide(pop_series, axis=0)


# Protein Supply per g per pax (y) 
# This dataset has more countries than the others
df_protein_agg = df_protein.loc[df_protein["Year"] == 2019]
# Need remove the (FAO) data as those are total for the whole continent
df_protein_agg = df_protein_agg.loc[~df_protein_agg["Country"].str.contains("FAO")]
srs_protein_agg = pd.Series(data=df_protein_agg["Food supply (Protein g per capita per day)"].values, index=df_protein_agg["Country"])

# Minor inconvience: If we check through protein-supply.csv, we see some countries that are misnamed/named on a different convention
# e.g "United States" instead of "United States of America" in the other data
# Check for list of missing names we need to replace
unique_country_names = srs_import_agg.index.append(srs_export_agg.index).append(srs_prod_agg.index).unique()
missing_names_mask = ~unique_country_names.isin(srs_protein_agg.index)
missing_names = unique_country_names[missing_names_mask]
print(missing_names)
# Now we can manually rename labels in srs_protein_agg that match the missing names using dictionary
srs_protein_agg.rename({"Taiwan": missing_names[0],
                        "South Korea": missing_names[1],
                        "United Kingdom": missing_names[3],
                        "United States": missing_names[4],
                        "Vietnam": missing_names[5]
                       }, inplace=True)
print(srs_protein_agg["Viet Nam"])

# Create new df with 12 columns including country
agg_data = [pop_series, srs_temp_agg, srs_cpi_agg, srs_gdp_cap, srs_import_agg, srs_export_agg, srs_prod_agg, srs_import_agg_1000, srs_export_agg_1000, srs_prod_agg_1000, srs_protein_agg]
col_headers = ["Country", "Population (per 1000)", "Temperature Change (degrees celsius)", "Consumer Price Index", "GDP per capita (USD millions)", "Import Quantity (tonnes)", "Export Quantity (tonnes)", "Production Quantity(tonnes)", "Import Quantity (tonnes per 1000 people)", "Export Quantity (tonnes per 1000 ppl)", "Production Quantity(tonnes per 1000 ppl)", "Protein Supply per g per pax (y)"]

df_aggregated = pd.concat(agg_data, axis=1)
df_aggregated = df_aggregated.reset_index()  # turn index which used to countries to its own column
df_aggregated = df_aggregated.replace(0, np.nan)  # to prepare to remove rows with 0.0 in data
df_aggregated = df_aggregated.dropna()  # remove rows/countries with incomplete data
df_aggregated.columns = col_headers
display(df_aggregated)

df_aggregated.to_csv("DDW Data/data_aggregated.csv", index=False)


Index(['China, Taiwan Province of', 'Republic of Korea', 'Singapore',
       'United Kingdom of Great Britain and Northern Ireland',
       'United States of America', 'Viet Nam'],
      dtype='object', name='Area')
88.84115


,Country,Population (per 1000),Temperature Change (degrees celsius),Consumer Price Index,GDP per capita (USD millions),Import Quantity (tonnes),Export Quantity (tonnes),Production Quantity(tonnes),Import Quantity (tonnes per 1000 people),Export Quantity (tonnes per 1000 ppl),Production Quantity(tonnes per 1000 ppl),Protein Supply per g per pax (y)
0,Afghanistan,37769.499,0.977167,115.988022,496.940553,53669.0,27.0,268273.0,1.420961,0.000715,7.102901,57.723488
1,Armenia,2820.602,1.904917,105.063042,4604.641377,64083.0,2086.0,107335.0,22.719618,0.739558,38.053933,98.883010
2,Australia,25357.170,1.499083,104.748284,54763.202388,275109.0,2495938.0,4818760.0,10.849357,98.431252,190.035402,107.283745
3,Bangladesh,165516.222,1.090083,125.605310,1846.416377,9326.0,71.0,707246.0,0.056345,0.000429,4.272971,60.414692
4,Belgium,11510.568,2.034917,106.583408,46388.205209,827415.0,1770220.0,1755460.0,71.883073,153.790847,152.508547,100.587840
5,Brazil,211782.878,1.578250,120.038862,8936.360101,61751.0,7559331.0,28578980.0,0.291577,35.693778,134.944714,93.485370
7,Canada,37522.584,1.242667,104.509242,46550.335507,709748.0,2068474.0,5114646.0,18.915222,55.126108,136.308470,108.227196
10,Colombia,50187.406,1.281250,120.441325,6424.979431,267765.0,27573.0,2900865.0,5.335303,0.549401,57.800656,72.939660
12,Croatia,4129.752,2.281000,103.325000,15070.610877,182208.0,62782.0,239861.0,44.120809,15.202366,58.081212,93.515686
13,Egypt,105618.671,1.022083,196.405120,3161.324684,728552.0,5673.0,2439520.0,6.897947,0.053712,23.097431,90.789450


<a id='step3'></a>
# Step 3: Choosing a model

Our chosen model is that of multiple linear regression.   

In our model, we hypothesize that the protein supplied to people in a country is represented by the function $\hat{y}$ as follows:

$$\hat{y}(x) =  \hat{\beta}_0 + \hat{\beta}_1 x_1 + \hat{\beta}_2 x_2 + \ldots + \hat{\beta}_n x_n$$  

#### <center> OR </center>  

$$\mathbf{\hat{y}} = \mathbf{X} \times \mathbf{\hat{b}}$$

The cost function of our model is the following, which is derived from the sum of squared errors, i.e $\frac{1}{2}$MSE:
$$J(\hat{\beta}_0, \hat{\beta}_1) = \frac{1}{2m}\Sigma^m_{i=1}\left(\hat{y}(x^i)-y^i\right)^2$$

Using calculus to minimize our cost J against $\hat{\beta}$, we can derive our gradient descent update equation as the following:
$$ \mathbf{\hat{b}} = \mathbf{\hat{b}} - \alpha\frac{1}{m} \mathbf{X}^T \times (\mathbf{X}\times \mathbf{\hat{b}} - \mathbf{y}) $$

With this, we get a column vector of $\hat{\beta}$ for our multiple linear regression model which best fits our data.  
  
To check if we have sufficiently minimized our cost, we can plot our cost at iteration, J, against iteration number.

In [ ]:
def calc_linear(X, beta):
    return np.matmul(X, beta)

def compute_cost(X, y, beta):
    J = 0
    m = X.shape[0]
    error = calc_linear(X, beta) - y
    error_sq = np.matmul(error.T, error)
    J = (1/(2*m))*error_sq
    J = J[0][0]
    return J

def gradient_descent(X, y, beta, alpha, num_iters):
    m = X.shape[0] 
    J_storage = np.zeros((num_iters, 1))
    
    for i in range(num_iters):
        beta = beta - (alpha / m) * np.matmul(X.T, (calc_linear(X, beta) - y))
        J_storage[i] = compute_cost(X, y, beta)
    return beta, J_storage

<a id='step4'></a>
# Step 4: Training

<a id='step5'></a>
# Step 5: Evaluation

Metrics wise there are 3 that works in linear regression (excluding their upgraded variations that work in multiple linear regression):
- R squared (Linear Regression only) —> Adjusted R squared (Both Single and Multiple Linear Regression)
- Mean Squared Error (MSE) —> Root Mean Squared Error (RMSE) (i.e just corrected for the squaring of error)
- Mean Absolute Error (MAE)

MSE, RMSE and MAE measure the error of the model against the actual data. All 3 metrics range from 0 to $\infty$, and is negatively-oriented (i.e the lower it is the better).
  
## R-Squared
R squared measures the fit of the model by explaining how much of a variation of a dependent variable is explainable by independent variable in regression model. R squared only works on single linear regression, as it increases or remains the same, making the model seem like a better fit than it actually is.

Instead, we use **Adjusted R-Squared** which adjusts for degree of freedom of data, and is unaffected by irrelevant predictors. 
$$ R^2_{\mathrm{adj}} = 1 - \frac{\mathrm{RSS}/(n-p)}{\mathrm{TSS}/(n-1)} $$

Adjusted R-Squared is positive-oriented (i.e higher is better fit), and ranges from 0 to 1.  

## Mean Squared Error
MSE and RMSE are the same in distribution, but RMSE is scaled down in value. 

Mean Squared Error = Mean of squared error = $ \frac{1}{n}\sum_{i=1}^{n}(\hat{y}(x^i)-y_i)^2 $  
Root Mean Squared Error = $ \sqrt{MSE} $
  
MSE and RMSE are sensitive to outliers. This is as it gives higher weight to larger errors (residuals), and is better used when large errors are particularly undesirable. 
* Since we desire to minimize RMSE, we want to over represent large errors.

This is sensitivity is due to the squaring of the errors, which we can see mathematically from how:
$$ (\frac{b}{a})^2 > \frac{b}{a} \text{, where b is the large error and a is the small error}$$


## Mean Absolute Error
MAE = Mean of absolute error = $\frac{1}{n}\sum_{i=1}^{n}|\hat{y}(x^i)-y_i|$
MAE treats every error equally and just finds their average.  

However, absolute values are undesirable in mathematics as absolute value is a piecewise function which are subject to both algebra and a geometric interpretation of piecewise graph.

$$ \lvert x \rvert = \left\{
        \begin{array}{ll}
            -x & \quad x < 0 \\
            x & \quad x \geq 0
        \end{array}
    \right. $$

## Choice of Metric: RMSE
Given our model of protein supply is worse off having larger errors due to outliers in the data, as well as the ease of mathematical calculations, we will be choosing RMSE as our main metric.

Adjusted R-Squared will be used to further evaluate the fit of our model.


References: 
* [Medium: RMSE vs MAE - Which metric is better](https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d#:~:text=RMSE%20has%20the%20benefit%20of,then%20MAE%20is%20more%20appropriate.)
* [Medium: Comparing robustness of MAE, MSE and RMSE](https://towardsdatascience.com/comparing-robustness-of-mae-mse-and-rmse-6d69da870828)

In [ ]:
def r2_score(y, ypred):
    y_mean = y.mean()
    ss_res = np.sum((y - ypred)**2)
    ss_tot = np.sum((y - y_mean)**2)
    return 1 - ss_res/ss_tot



def mean_squared_error(target, pred):
    n = target.shape[0]
    rss = np.sum((target - pred)**2)
    mse = rss / n
    return mse

def root_mean_squared_error(target, pred):
    return sqrt(mean_squared_error(target, pred))



<a id='step6'></a>
# Step 6: Parameter Tuning
Predicted vs actual graphs so can tell relationships and evalute model.  
  
## Residual plots
Residual = actual y - predicted y  
  
Residual Plot is a plot of predicted y against standardized residual.

# Hyperparameters to change
* Alpha in gradient descent - i.e steps  
* Iterations in gradient descent  
* Starting guesses for $\hat{b}$, the column vector containing $\beta_i$

<a id='step7'></a>
# Step 7: Prediction

Use model to guess protein supply per capita given a set of predictor values.
  
Covariance of the protein supply against the different predictors can help us determine the relationship of the predictor to the protein supply.

<a id='modelimp'></a>
# Model Improvements
## Improvement 1: Removal of outliers
From our data, we can see certain countries protein supply are far from the median.  
Examples include: .... countries ....   
  
We can arbitarily define outliers as protein supply values less than (Q1 - 1.5 IQR) or protein supply values more than (Q3 + 1.5 IQR). The functions to find these are defined below.
  
This has two implications that allows us to get a better model:  
1. Our chosen cost metric, RMSE, is more sensitive to extreme values. By removing outliers, we can lower our RMSE.
2. Our cost function J is equal to $\frac{1}{2}$ MSE which like RMSE is sensitive to outliers.

We can also plot the outliers with a box and whiskers plot.

# Improvement 2: Removal of irrelevant / low correlation predictors  
  
Based on our graphs, we will be removing ________ as our predictor(s).  


In [ ]:
def quartile_1(df, target):
    return

def quartile_3(df, target):
    return

def iqr(target):
    return quartile_3(df, target) - quartile_1(df, target)

def remove_outliers(df, target):
    """ 
    Returns a new dataframe with the outliers row removed.  
    """
    return df_new

